# wordextractor

In [1]:
import sys
sys.path.insert(0, '../')
from soynlp.utils import DoublespaceLineCorpus
    
corpus_fname = '1118_shampoo_review_corpus.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
print('num sentences = %d' % len(sentences))

num sentences = 58547


### train은 substrings의 빈도수를 카운팅 하는 것이며, extract는 init에 들어가는 값을 기준으로 단어를 선택하여 줍니다.

In [4]:
%%time

from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    min_frequency=2, # 수정된 부분 빈도수 최소 2번으로 바꿈. 그전에 5번이었음.
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.321 Gbory 0.285 Gb
all cohesion probabilities was computed. # words = 107889
all branching entropies was computed # words = 71326
all accessor variety was computed # words = 71326
CPU times: user 12.3 s, sys: 104 ms, total: 12.4 s
Wall time: 12.4 s


In [5]:
len(words)

56577

In [6]:
result = []
for key in words:
    if 1 <= len(key):
        result.append(key)
         
#print(result)

In [7]:
len(result)

56577

### 필요 단어를 리스트로 다 만듭니다. 
### if 조건에서 본인 담당한 필터의 워드를 넣으시면 됩니다. soynlp가 위에서 빈도수 학습한 걸로 조건에 들어간 글자 어절을 뽑아줘요 

In [8]:
result_2 = []
for word in result:
    if '두피' in word or '자극' in word or'건조' in word or '비듬' in word or '기름' in word \
    or'떡' in word or'트러블' in word or'간지' in word or'가려' in word:
        result_2.append(word)


stopword_sample = result_2
len(stopword_sample)

2361

## 여기서 부터 ngram 리뷰 칼럼 만들기 

In [11]:
import pandas as pd

In [12]:
# 데이터 불러오세요. 

data = pd.read_excel('1118_review_preprocessing.xlsx')

In [13]:
data.drop('Unnamed: 0', axis = 'columns', inplace=True)

In [14]:
data.head(1)

,brand,product,review,거품_카테고리,거품,두피,머릿결,세정력_카테고리,세정력,자극도_카테고리,탈모,향_카테고리,향,label
0,비욘드,힐링포스 프로페셔널 스캘프 샴푸,이거 진심 대박템임 어렸을 때는 숱 많고 굵은 모발이었던 나 그러나 자라면서 머리카...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train


### 본인 담당의 ngram 칼럼을 정해주세요. 칼럼 이름은 본인이 담당한 'split_(본인 담당 필터)' 으로 통일할게요 

In [15]:
data['split_target'] = '' # 예시 칼럼 이름입니다. 

### ngram 칼럼 추출

## logic 테스트 코드 - 실행 안해도 됩니다. 이해를 위해 실험한 코드입니다. 
#### 이 코드는 ngram test를 위한 테스트 코드입니다. 이해를 원하는 분들을 위해 남겨둘테니 한 번 실행 해보면서 이해해보세요 

In [197]:
# 로직 테스트 

split_review = data['review'].iloc[1,].split()

target_list = []

for word in split_review:
        if word in stopword_sample:
            target_list.append(split_review.index(word))
            
print(target_list) # target의 index 위치 리스트 


sample_list = []

for target in target_list:
    if target < 3:
         sample_list.append(' '.join(split_review[:target+4]) + '\n')
    else:
        sample_list.append(' '.join(split_review[target-3:target+4]) + '\n')

print('sample_str 출력: \n' , sample_str)
        
for ngram in sample_list:
    text = ngram.split()
    for word in text:
        result.append(word) 

result2 = list(OrderedDict.fromkeys(result))       
        
        
print(sample_list)
print(str(sample_list))
print(result2)
print(' '.join(result2))

[8, 47, 48, 84]
sample_str 출력: 
 사러 갔습니다 전 두피 정말 민감해서 제일
향이 무지 좋고 두피에 자극되는 느낌이 정말
무지 좋고 두피에 자극되는 느낌이 정말 없더라구요
샴푸 써봐도 다 간지럽고 긁게 됐는데 왠지

['사러 갔습니다 전 두피 정말 민감해서 제일\n', '향이 무지 좋고 두피에 자극되는 느낌이 정말\n', '무지 좋고 두피에 자극되는 느낌이 정말 없더라구요\n', '샴푸 써봐도 다 간지럽고 긁게 됐는데 왠지\n']
['사러 갔습니다 전 두피 정말 민감해서 제일\n', '향이 무지 좋고 두피에 자극되는 느낌이 정말\n', '무지 좋고 두피에 자극되는 느낌이 정말 없더라구요\n', '샴푸 써봐도 다 간지럽고 긁게 됐는데 왠지\n']
['사러', '갔습니다', '전', '두피', '정말', '민감해서', '제일', '향이', '무지', '좋고', '두피에', '자극되는', '느낌이', '없더라구요', '샴푸', '써봐도', '다', '간지럽고', '긁게', '됐는데', '왠지']
사러 갔습니다 전 두피 정말 민감해서 제일 향이 무지 좋고 두피에 자극되는 느낌이 없더라구요 샴푸 써봐도 다 간지럽고 긁게 됐는데 왠지


In [155]:
# 테스트용 코드 

from collections import OrderedDict

result = []

for ngram in sample_list:
    text = ngram.split()
    for word in text:
        result.append(word) 

result2 = list(OrderedDict.fromkeys(result)) # 순서 유지하면서 중복 제거 

print(result) # 중복 제거하기전 
print(result2) # 중복 제거 

['사러', '갔습니다', '전', '두피', '정말', '민감해서', '제일', '향이', '무지', '좋고', '두피에', '자극되는', '느낌이', '정말', '무지', '좋고', '두피에', '자극되는', '느낌이', '정말', '없더라구요', '샴푸', '써봐도', '다', '간지럽고', '긁게', '됐는데', '왠지']
['사러', '갔습니다', '전', '두피', '정말', '민감해서', '제일', '향이', '무지', '좋고', '두피에', '자극되는', '느낌이', '없더라구요', '샴푸', '써봐도', '다', '간지럽고', '긁게', '됐는데', '왠지']


## 리스트 중복 제거 - 순서 살리고 싶을 경우 아래 라이브러리 사용 
- 알아두면 유용한 방법입니다. 

https://bit.ly/37c9vUN

In [154]:
from collections import OrderedDict

originalList = [1, 2, 3, 1, 2, 5, 6, 7, 8]
list(OrderedDict.fromkeys(originalList))

[1, 2, 3, 5, 6, 7, 8]

In [177]:
text = ''
text = text + ' '.join(split_review) + ' '+ ' '.join(split_review) 
print(text)

#text.split()

성분 좋은 아로마티카라서 믿고 삽니다 아로마티카 다른 샴푸들도 사용해보고 다 좋았어서 이 제품도 사봤어요 일단 성분이 좋으니 믿고 구매하는 아로마티카입니다  성분 좋은 아로마티카라서 믿고 삽니다 아로마티카 다른 샴푸들도 사용해보고 다 좋았어서 이 제품도 사봤어요 일단 성분이 좋으니 믿고 구매하는 아로마티카입니다


### 여기 밑에서부터 다시 실행하시면 되요

## 최종 ngram 추출 코드 
- 제 노트북은 5분 정도 걸렸던 거 같아요.

In [16]:
# 두피 ngram 추출 
# 최종 결과 string 으로 출력 - 두 번째 시도 

from collections import OrderedDict

for i, review in enumerate(data['review']):
    if type(review) != float:
        target_list = []
        split_review = review.split()

        for word in split_review:
            if word in stopword_sample:
                target_list.append(split_review.index(word))
        
        sample_list = []
        
        for target in target_list:
                if target < 3:
                    sample_list.append(' '.join(split_review[:target+4]) + '\n')
                else:
                    sample_list.append(' '.join(split_review[target-3:target+4]) + '\n')
                    
        result = []
        
        for ngram in sample_list:
            text = ngram.split()
            for word in text:
                result.append(word) 

        result2 = list(OrderedDict.fromkeys(result))  # 순서 유지하면서 중복 제거      
        result3 = ' '.join(result2) + '\n'
        data['split_target'][i] = result3 

/home/daehynk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
# 잘 들어갔는지 확인 

count = 0 

for split in data['split_target']:
    if len(split) >1:
        print(split)
        count += 1
        if count == 15:
             break

사러 갔습니다 전 두피 정말 민감해서 제일 향이 무지 좋고 두피에 자극되는 느낌이 없더라구요 샴푸 써봐도 다 간지럽고 긁게 됐는데 왠지

엄청 시원하진 않고 두피 시윈한걸로 치면 이니스프리 뭔가 몸에 좋은냄새한방허브느낌 두피관리 기능은 잘 모르겠음원래도

전에 쫌 냅두면 두피가 화아 하니 박하

하자면탈모에도 좋다고 함 두피나 머리결어 따라서 맞추어 더 컷더라면요거는 시원하게혹은 떡지는거 싫으신분들이 좋아하실듯생각보다 시원했고산뜻하기도 그냥 막 쓰기보다 두피에 좋은 그나마나 괜찮은

탈모 원인중에 하나가 두피라고 하던데 저는 두피가 붉고 약한상태였거든요 근데 두피상태다 좋아져서 지금은 두피건강도 좋아졌어요 이거 살때 사은품으로 주셨던 두피 마사지 빗도 있는데

지성두피용 탈모가는모발 두피입니다 보라색통 지성용 프로패셔널 엉키니까 컨디셔너 써야되고 두피나 등에 여드름 나요

너무 좋다 두피케어제품 오만가지 다 쓰지만

뾰루지같은게 생기기 시작하고 기름도 지는 것 같아서 좋고 거품도 풍성하고 떡지거나 가렵거나 뾰루지 나지도

다른샴푸랑 똑같이 행구는데도 두피에 미끌미끌한게 남음 아무리헹궈도

향이구요 저는 쓰면서 두피가 간지럽거나 냄새가 난다던가 안좋은

감고 나면 머리 가려움이 덜함 떡지는 것도 평타이상은

전 두피가 건조한 분들께 추천한다는 리뷰가 제 지인도 사용해보니까 건조하다고 하더라구요 거품 은

향은 별로라 여름 두피에 땀 흘릴때 머리속 지루성 두피염으로 고생인데 그것까지는 잡아주지는

거품 잘 나고 비듬도 안 생기고 머릿결도

거품 잘나는편이고 거품이 두피에 닿으면 아무 느낌 살짝 화해지는데 정작 두피가 시원해지지는 않았다 단점이



## 저장 - '날짜_split_review_자기담당필터_본인 이름.xlsx' 로 통일할게요  

In [19]:
data.to_excel('1119_split_review_두피.xlsx')

In [18]:
# 두피 ngram 개수 확인 

length = 0
count = 0

for i in range(len(data)):
    if len(data['split_target'].iloc[i,]) > 1:
        count  += 1
        length +=1
        
print(length)

12165
